In [5]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import pickle
from sklearn.preprocessing import MinMaxScaler
from tensorflow.keras.layers import Masking, GRU, Dense, Dropout
from tensorflow.keras.models import Sequential
from tensorflow.keras.optimizers import Adam
import numpy as np # linear algebra
import pandas as pd
import random
import matplotlib.pyplot as plt
import os
from glob import glob
from sklearn.preprocessing import StandardScaler , LabelEncoder
from tqdm import tqdm

In [6]:
features = ['Delta','Theta','Alpha1','Alpha2','Beta1','Beta2','Gamma1','Gamma2', 'Attention','Meditation']

def create_input_array(df_grouped, features):
    X_list = []
    for idx, row in df_grouped.iterrows():
        sample = np.stack([row[feat] for feat in features], axis=1)
        X_list.append(sample)
    X = np.array(X_list)
    return X


file = "/kaggle/input/superai5-pain-syndrome-classification/X_test.pkl"
with open(file, 'rb') as f:
    X_train = pickle.load(f)
X_train = create_input_array(X_train, features)


In [107]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense, Dropout, Conv1D, MaxPooling1D, Input
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.losses import SparseCategoricalCrossentropy
from tensorflow.keras.callbacks import ModelCheckpoint

In [329]:
features = ['Delta','Theta','Alpha1','Alpha2','Beta1','Beta2','Gamma1','Gamma2', 'Attention','Meditation']

def create_input_array(df_grouped, features):
    X_list = []
    for idx, row in df_grouped.iterrows():
        sample = np.stack([row[feat] for feat in features], axis=1)
        X_list.append(sample)
    X = np.array(X_list)
    return X


file = "/kaggle/input/superai5-pain-syndrome-classification/X_train.pkl"
with open(file, 'rb') as f:
    X_train = pickle.load(f)
X_train = create_input_array(X_train, features)

file = "/kaggle/input/superai5-pain-syndrome-classification/X_test.pkl"
with open(file, 'rb') as f:
    X_test = pickle.load(f)
X_test = create_input_array(X_test, features)

file = "/kaggle/input/superai5-pain-syndrome-classification/y_train.csv"
y_train = pd.read_csv(file).set_index('person_id')


def build_model_gru(input_shape, num_classes): # Drop out ชอบตอบ 0 ไม่ดี ต้องมี 0 มากกว่าครึ่ง fully ไม่ค่อยดี
    model = Sequential()
    model.add(Masking(mask_value=0., input_shape=input_shape))

    # model.add(Conv1D(filters=64, kernel_size=2, activation='relu', padding='same'))
    
    model.add(GRU(32, return_sequences=True)) #GRU 64 or 32
    # model.add(Dropout(0.5))
    # model.add(GRU(64, return_sequences=False))
    # model.add(Dropout(0.5))
    # model.add(Conv1D(filters=64, kernel_size=2, activation='relu', padding='same'))
    model.add(LSTM(32, return_sequences=False)) #none orr 64 or 32
    # model.add(Dropout(0.1))
    # model.add(Dense(16, activation="gelu"))
    # # model.add(Dropout(0.2))
    # model.add(Dense(4, activation="relu"))

    
    model.add(Dense(num_classes, activation='softmax'))
    model.compile(optimizer=Adam(learning_rate=0.01),
                  loss='sparse_categorical_crossentropy',
                  # loss='crossentropy',
                  metrics=['accuracy'])
    return model
# def build_model_gru(input_shape, num_classes):
#     model = Sequential()
#     # model.add(Input(shape=input_shape))
#     model.add(Masking(mask_value=0., input_shape=input_shape))
    
#     # model.add(Dense(256, activation="relu"))
#     # model.add(Dropout(0.3))
#     # model.add(Dense(128, activation="relu"))
#     # model.add(Dropout(0.3))
#     # model.add(Dense(64, activation="relu"))

#     # Fully connected layers for classification
#     # model.add(Dense(32, activation="relu"))
#     # model.add(Dropout(0.3))
#     # model.add(Dense(32, activation="relu"))
#     model.add(Dense(num_classes, activation="softmax"))
    
#     model.compile(optimizer=Adam(learning_rate=0.001),
#                   loss='sparse_categorical_crossentropy',
#                   metrics=['accuracy'])
#     return model


num_classes = 4
input_shape = X_train.shape[1:]

scaler = MinMaxScaler()

X_train_reshaped = X_train.reshape((-1, input_shape[-1]))
X_train_scaled = scaler.fit_transform(X_train_reshaped)
X_train = X_train_scaled.reshape((-1, input_shape[0], input_shape[-1]))

X_test_reshaped = X_test.reshape((-1, input_shape[-1]))
X_test_scaled = scaler.transform(X_test_reshaped)
X_test = X_test_scaled.reshape((-1, input_shape[0], input_shape[-1]))

model = build_model_gru(input_shape, num_classes)

epochs = 20 # 20-300
batch_size = 16 

model.fit(X_train, y_train.values, epochs=epochs, batch_size=batch_size, verbose=1)

y_pred = model.predict(X_test).argmax(axis=1)
y_pred

Epoch 1/20


/usr/local/lib/python3.10/dist-packages/keras/src/layers/core/masking.py:47: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


5/5 ━━━━━━━━━━━━━━━━━━━━ 2s 68ms/step - accuracy: 0.3045 - loss: 1.3556
Epoch 2/20
5/5 ━━━━━━━━━━━━━━━━━━━━ 0s 62ms/step - accuracy: 0.3688 - loss: 1.2319
Epoch 3/20
5/5 ━━━━━━━━━━━━━━━━━━━━ 0s 61ms/step - accuracy: 0.3263 - loss: 1.3081
Epoch 4/20
5/5 ━━━━━━━━━━━━━━━━━━━━ 0s 62ms/step - accuracy: 0.3984 - loss: 1.2815
Epoch 5/20
5/5 ━━━━━━━━━━━━━━━━━━━━ 0s 62ms/step - accuracy: 0.3280 - loss: 1.2711
Epoch 6/20
5/5 ━━━━━━━━━━━━━━━━━━━━ 0s 63ms/step - accuracy: 0.4538 - loss: 1.2319
Epoch 7/20
5/5 ━━━━━━━━━━━━━━━━━━━━ 0s 62ms/step - accuracy: 0.5230 - loss: 1.2141
Epoch 8/20
5/5 ━━━━━━━━━━━━━━━━━━━━ 0s 63ms/step - accuracy: 0.3667 - loss: 1.2967
Epoch 9/20
5/5 ━━━━━━━━━━━━━━━━━━━━ 0s 64ms/step - accuracy: 0.4773 - loss: 1.2554
Epoch 10/20
5/5 ━━━━━━━━━━━━━━━━━━━━ 0s 62ms/step - accuracy: 0.4289 - loss: 1.3098
Epoch 11/20
5/5 ━━━━━━━━━━━━━━━━━━━━ 0s 64ms/step - accuracy: 0.4362 - loss: 1.2223
Epoch 12/20
5/5 ━━━━━━━━━━━━━━━━━━━━ 0s 63ms/step - accuracy: 0.4630 - loss: 1.2526
Epoch 13/20


array([0, 0, 0, 0, 0, 1, 0, 1, 1, 0, 0, 0, 1, 0, 0, 0, 0])

In [330]:
sample = pd.read_csv("/kaggle/input/superai5-pain-syndrome-classification/sample_submission.csv")
sample['pain_version'] = y_pred
sample

,person_id,pain_version
0,S021,0
1,S093,0
2,S066,0
3,S056,0
4,S032,0
5,S046,1
6,S027,0
7,S031,1
8,S012,1
9,S037,0


In [331]:
sample.to_csv("submission.csv",index=False)